# Technical challenge

### Mode 1 - Tagging image in order given by .json data

In [628]:
# Import needed modules
from PIL import Image, ImageDraw
import json
import matplotlib.colors as mcolors

# Loading image and coresponding .json file
img = Image.open("samples/18.jpg")
json_data = json.load(open('samples/18.json'))


# Create a new image to be used as a new image output
draw = ImageDraw.Draw(img)

# Browsing dictionary items for image tagging information
for item in json_data["ocr"]: 
    text = item['text']
    
    # Getting coordinates of upper left corner and lower right corner of rectangle
    x1 = item['points'][0][0]
    y1 = item['points'][0][1]
    
    x2 = item['points'][1][0]
    y2 = item['points'][2][1]
    
    # Getting the color pallet list of CSS4
    colors = list(mcolors.CSS4_COLORS.keys()) 
    
    # Create a color range for each line with posibility of adjustment (line_tollerance) for different images
    middle_line = (y1+y2) / 2
    line_tollerance = 15
    color_index = int((img.height-middle_line)/line_tollerance)
    
    # Tagging the image with rectangles and writing text in a distinct color range
    draw.rectangle([x1,y1,x2,y2],width=3,outline=colors[color_index])
    draw.text([x1, y1-10], text, fill=colors[color_index])
img.show()
img.save('output.jpg')


img.close()

### Mode 2 - Tagging after data sorting

In [631]:
# Import needed modules
from PIL import Image, ImageDraw
import json
import matplotlib.colors as mcolors
import pandas as pd
from pandas.io.json import json_normalize

path_image = "samples/40.jpg"
path_json = "samples/40.json"

with open(path_json,'r') as f:
    data = json.loads(f.read())

# Create Data Frame with content of .json file
df = pd.json_normalize(data, record_path =['ocr'])


data_x1 = []
data_y1 = []
data_x2 = []
data_y2 = []

# Extract coordinates of rectangle points for each word
for data in df['points']:
    data_x1.append(data[0][0])
    data_y1.append(data[0][1])
    data_x2.append(data[1][0])
    data_y2.append(data[2][1])
        
df['x1'] = data_x1
df['y1'] = data_y1
df['x2'] = data_x2
df['y2'] = data_y2

#Adding coordinates to Data Frame
df = df[['text','x1','y1','x2','y2']]

df=df.sort_values(['y1','x1'])

# Loading image and coresponding .json file
img = Image.open(path_image)

# Create a new image to be used as a new image output
draw = ImageDraw.Draw(img)

for item in df.values:
    text = item[0]
    
    # Getting coordinates of upper left corner and lower right corner of rectangle
    x1 = item[1]
    y1 = item[2]
    
    x2 = item[3]
    y2 = item[4]
    
    # Getting the color pallet list of CSS4
    colors = list(mcolors.CSS4_COLORS.keys()) 
    
    # Create a color range for each line with posibility of adjustment (line_tollerance) for different images
    middle_line = (y1+y2) / 2
    line_tollerance = 30
    color_index = int((img.height-middle_line)/line_tollerance)
    
    # Tagging the image with rectangles and writing text in a distinct color range
    draw.rectangle([x1,y1,x2,y2], width=3, outline=colors[color_index])
    draw.text([x1, y1-10], text, fill=colors[color_index])
img.show()
img.save('output.jpg')
img.close()